#### Imports

In [542]:
import PyPDF2
import csv
import re
import json
import os
import us
import numpy as np

#### Scrape key to facility codes
- Very inclusive info about keys, many things not keys included

In [3]:
year = 1998

pdfFileObj = open('{0}/directory_{0}.pdf'.format(year), 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 568


In [582]:
abbr_keys = ["TX", "ML", "PV", "OS", "DT",\
             "OD", "OR", "IO", "ID", "IR", "RD", "RR",\
             "AD", "DD", "HV", "PW",\
             "SS", "GH", "PH", "OH", "TC", "HH", "CM", "CH", \
             "CO", "SG", "SC", "CJ",\
             "MC", "MD", "FG", "PI"]

#### List of States in the U.S.

In [476]:
states = "Alabama,\
Alaska,\
Arizona,\
Arkansas,\
California,\
Colorado,\
Connecticut,\
Delaware,\
Florida,\
Georgia,\
Hawaii,\
Idaho,\
Illinois,\
Indiana,\
Iowa,\
Kansas,\
Kentucky,\
Louisiana,\
Maine,\
Maryland,\
Massachusetts,\
Michigan,\
Minnesota,\
Mississippi,\
Missouri,\
Montana,\
Nebraska,\
Nevada,\
New Hampshire,\
New Jersey,\
New Mexico,\
New York,\
North Carolina,\
North Dakota,\
Ohio,\
Oklahoma,\
Oregon,\
Pennsylvania,\
Rhode Island,\
South Carolina,\
South Dakota,\
Tennessee,\
Texas,\
Utah,\
Vermont,\
Virginia,\
Washington,\
West Virginia,\
Wisconsin,\
Wyoming"

In [477]:
states = states.split(",")

In [478]:
states

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

#### Test on a page of data
- data starts on page 12 (getPage(11))
- split to each city, then split into data centers, clean city name and keys!

- Function to match city names

In [888]:
def fuzzy_search(search_key, text):
    #print("Searching {}...".format(search_key))
    #print("in", text)
    l = len(search_key)
    search_key = search_key.lower()
    strictness = int(l/4) + 1
    for pos in re.finditer(r"[\.,]", text):
        word_to_test = text[pos.start() - l: pos.start()].lower()
        i = 0
        diff = 0
        while i < l:
            if word_to_test[i] != search_key[i]:
                diff += 1
            i += 1
        if diff < strictness:
            return (pos.start() - l, pos.start())      

In [889]:
#test
text = "DemopolisWestAl.abamaMentalDcmopolis.AL36732"
fuzzy_search("DEMOPOLIS", text)

(27, 36)

- Fucntion to split words

In [985]:
def split_cap_words(string):
    string = re.sub(r"([A-z()])([A-Z0-9])([a-z0-9])", r"\1 \2\3", string)
    string = re.sub(r"([0-9])([A-Za])", r"\1 \2", string) #keep 1st, 2nd, 3rd, 4th
    string = re.sub(r"([A-z])([0-9])", r"\1 \2", string)
    string = re.sub(r"([A-z])(of)( )", r"\1 \2\3", string)
    return  string#(\w)([A-Z])([a-z])

In [986]:
#test
split_cap_words("Ltghthouseof Tallapoosa County Inc")

'Ltghthouse of Tallapoosa County Inc'

In [987]:
split_cap_words("17136thAvenueSouth")

'17136th Avenue South'

In [988]:
split_cap_words("17132ndAvenueSouth")

'17132nd Avenue South'

In [989]:
split_cap_words("PGBHAPinalCountyHispanicCouncil712NorthMainStreet")

'PGBHA Pinal County Hispanic Council 712 North Main Street'

In [990]:
split_cap_words("CommunityCareNetwork(CCN)JewishFamilyandChildrensServices6376WestBellRoad")

'Community Care Network(CCN) Jewish Familyand Childrens Services 6376 West Bell Road'

In [991]:
split_cap_words("MohaveMentalHealthClinicOutpatientandDayTreatment2135Highway9SSuites125and241")

'Mohave Mental Health Clinic Outpatientand Day Treatment 2135 Highway 9 S Suites 125 and 241'

In [992]:
split_cap_words("564NorthIdahoRoadSuite9")

'564 North Idaho Road Suite 9'

- Split data into a list containing a list of facilities in that city

In [1028]:
def split_data(rawdata, firstcity):
    '''
    Roughly split data into each facility, returns list of list.
    '''
    
    data = []
    city = firstcity
    
    while len(rawdata) > 50:
        temp_city = []
        m = 1
        
        if rawdata[:3].isupper(): #find city name and delete from rawdata
            #find the capitalized string
            m = re.search("[A-Z]+", rawdata).end()
            city = rawdata[:m - 1]
            #print("Trying to find city...", city)
            #clean state name from the city name if present
            for state in states:
                if state.lower() in city.lower():
                    city = city[len(state):]
                    rawdata = rawdata[len(state):]
                    m = m - len(state)
            #identify city name
            if fuzzy_search(city, rawdata[m - 1:]) == None:
                i = 2
                while (i < m) and (fuzzy_search(rawdata[:i], rawdata[i:]) == None):
                    i += 1
                m = i + 1
                city = rawdata[:i]
                
        rawdata = rawdata[m - 1:]
        a, b = fuzzy_search(city, rawdata)
        state_clean = split_cap_words(rawdata[a:b])
        temp_city.append(state_clean)
        #while there is more facilities in this city (still can find city name in the string)
        while fuzzy_search(city, rawdata) != None:
            temp_facil = []
            a, b = fuzzy_search(city, rawdata)
            m = re.search("[A-Z]{2}[A-Z/][A-Z/]", rawdata[b:]).start() #start of the keys
            temp_facil.append(rawdata[:b+m])
            rawdata = rawdata[b+m:]
            temp_facil.append('')
            j = 0
            #append keys to the dataframe
            while (rawdata[j:j+2].replace("1", "I").replace("l", "I").replace("0", "O").replace("5", "S")\
                   .replace("i", "I") in abbr_keys) and (rawdata[j+2].replace("1", "I").replace("l", "I")\
            .replace("0", "O").replace("5", "S").replace("i", "I").isupper() or rawdata[j+2] == '/'):
                temp_facil[-1] += ' ' + \
                rawdata[j:j+2].replace("1", "I").replace("l", "I").replace("0", "O").replace("5", "S")
                if rawdata[j+2] == "/" or rawdata[j+2] == "J": #"/" is sometimes recognized as "J"
                    j += 3
                else:
                    j += 2
                    
            rawdata = rawdata[j:]
            temp_facil[-1] = temp_facil[-1].strip()
            temp_city.append(temp_facil)
            #print(temp_facil)#####
        data.append(temp_city)
        i = 0
    
    while i < len(data):
        city = data[i]
        if not city[0][0].isupper():
            init = data[i - 1][-1][-1][-2:].capitalize()
            data[i - 1][-1][-1] = data[i - 1][-1][-1][:-3]
            data[i][0] = init + data[i][0]
        i += 1
    return data

In [1022]:
def split_center_street(cn_sa):
    '''
    Split a string containing center name and street address into two.
    Inputs:
        cn_sa(str)
    Returns: a list of two terms
    '''
    
    m = re.finditer(r"\d+", cn_sa)
    try:
        probe = next(m)
        span = probe.span()
        if span[1] == len(cn_sa):
            cn_sa = split_cap_words(cn_sa).split()
            return[' '.join(cn_sa[:-2]), ' '.join(cn_sa[-2:])]
        elif span[0] != 0:
            return ([split_cap_words(cn_sa[:span[0]]), split_cap_words(cn_sa[span[0]:])])
    except StopIteration:
        if cn_sa[-4:] == "Road":
            cn_sa = split_cap_words(cn_sa).split()
            return[' '.join(cn_sa[:-2]), ' '.join(cn_sa[-2:])]
    return [split_cap_words(cn_sa), '']


def split_address(csp):
    '''
    Split a string containing city, state and postal code.
    Inputs:
        csp(str): a string containing city, state and postal code
    Returns: a list of three items
    '''
    PASS

In [924]:
cn_sa = "SMMHCIncMentalHealthCtr/SubstAbuseServices564NorthIdahoRoadSuite9"
split_center_street(cn_sa)

['SMMHC Inc Mental Health Ctr/Subst Abuse Services',
 '564 North Idaho Road Suite 9']

In [925]:
cn_sa = "FortMcDowellIndianCommunityFamilyandCommunityServicesYavapaiRoad"
split_center_street(cn_sa)

['Fort McDowell Indian Community Familyand Community Services', 'Yavapai Road']

In [926]:
cn_sa = "VentressCorrectionalFacilitySubstanceAbuseServicesStateRoad239"
split_center_street(cn_sa)

['Ventress Correctional Facility Substance Abuse Services State', 'Road 239']

In [927]:
def write_data(writer, data, year, pagen):
    '''
    Write data to csv
    '''
    for city in data:
        cityname = city[0]
        lencn = len(cityname.replace(' ', ''))
        for facil in city[1:]:
            data_holder = [year, pagen]
            keys = facil[1]
            facil = facil[0]
            #Center name
            if "," in facil:
                center_name_city, address = facil.split(",")
            else:
                center_name_city, address = facil.split(".")
            #City
            data_holder += split_center_street(center_name_city[:-lencn])
            #State
            data_holder.append(cityname)
            data_holder.append(address[:2])
            #Postal code, Phone
            if address[7] == "-":
                data_holder.append(address[2:12].replace("O", "0").replace("S", "5"))
                data_holder.append(address[12:].replace("O", "0").replace("S", "5"))
            else:
                data_holder.append(address[2:7].replace("O", "0").replace("S", "5"))
                data_holder.append(address[7:].replace("O", "0").replace("S", "5"))
            #Keys
            data_holder.append(keys)
            print(data_holder)
            writer.writerow(data_holder)
    return data_holder[4]

In [928]:
def run(year, start_page, end_page = 0):

    #read file
    dir_filename = '{0}/directory_{0}.pdf'.format(year)
    pdfFileObj = open(dir_filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    #pages to scrape
    if end_page == 0:
        end_page = pdfReader.numPages - 1

    save_as_filename = '{0}/{0}.csv'.format(year)
    if os.path.isfile(save_as_filename):
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
    else:
        with open(save_as_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", \
                             "State", "Postal_code", "Phone", "Keys"])
    #scrape
    firstcity = "Alexandar City"
    for pagen in range(start_page, end_page + 1):
        #print(pagen)
        rawdata = pdfReader.getPage(pagen - 1).extractText()
        
        data = split_data(rawdata, firstcity)
        
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
            firstcity = write_data(writer, data, year, pagen)

In [1013]:
run(1998, 15, 561) #12 15

[1998, 15, 'Akeela Treatment Services', '2805 Bering Street Suite 4', 'Anchorage', 'AK', '99503', '(907)561-5266', 'TX PV OS OR RR SS TC SC CJ MD']
[1998, 15, 'Alaska Human Services Inc Outpt Alc/Subst Abuse Treatment Prog', '4050 Lake Otis Parkway Suite 111', 'Anchorage', 'AK', '99508', '(907)561-4535', 'TX OS OR IO SS PI']
[1998, 15, 'Alaska North Addictions Recovery Ctr', '4330 Bragaw Street', 'Anchorage', 'AK', '99508', '(907)561-5537', 'TX PV OS OR IO RR SS MD FG PI']
[1998, 15, 'Booth Memorial Youthand Family Servs', '3600 East 20th Avenue', 'Anchorage', 'AK', '99508', '(907)279-0522', 'TX PV OS OR IO AD DD PW SS MD']
[1998, 15, 'Charter North Behavioral Health System', '2530 Debarr Road', 'Anchorage', 'AK', '99508', '(907)258-7575Hotlines:(907)258-7575(800)478-7575', 'TX PV OS DT ID IR AD PH']
[1998, 15, 'Genesis House Inc', '2825 West 42nd Place', 'Anchorage', 'AK', '99517', '(907)243-5130', 'TX PV OS DT OR IO RD RR DD SS CM FG PI']
[1998, 15, 'Narcotic Drug Treatment Center In

[1998, 21, 'New Lifefor Girls', '6216 North 27th Avenue', 'Phoenix', 'AZ', '85017', '(602)246-2704', 'TX OS DT RD RR SS TC CO']
[1998, 21, 'PheonixLARC Meta Center Public Inebriate Program', '2770 East Vanburen Street', 'Phoenix', 'AZ', '85008-6039', '(602)273-9999Hotlines:(602)222-9444(800)631-1314', 'TX DT RD RR SS CH MD']
[1998, 21, 'Phoenix Indian Center', '2601 North 3rd Street Suite 100', 'Phoenix', 'AZ', '85004', '(602)263-1017', 'TX OS OR CM MD']
[1998, 21, 'Progress Valley Phoenix', '4430 North 23rd Avenue', 'Phoenix', 'AZ', '85015', '(602)274-5424', 'TX OS RR DD PW SS TC']
[1998, 21, 'Saint Lukes Behavioral Health Center Chemical Dependency Treatment', '1800 East Van Buren Street', 'Phoenix', 'AZ', '85006', '(602)251-8484Hotline:(602)251-8535', 'TX PV OS DT OD OR IO ID IR AD DD SS PH MC MD FG PI']
[1998, 21, 'Saint Lukes Behavioral Health Center', '1800 East Van Buren Street', 'Phoenix', 'AZ', '85006', '(602)251-8535Hotline:(602)251-8535', 'TX PV OS DT OD OR IO ID IR AD DD SS

TypeError: cannot unpack non-iterable NoneType object

In [842]:
year = 1998
pagen = 22
firstcity = "Montgomery"
rawdata = pdfReader.getPage(pagen - 1).extractText()
data = split_data(rawdata, firstcity)
with open(save_as_filename, 'a', newline='') as file:
    writer = csv.writer(file)
    clean_data(writer, data, year, pagen)

In [1014]:
pagen = 23
rawdata = pdfReader.getPage(pagen - 1).extractText()
rawdata

'CODACBehavioralHealthServicesofPimaCountyInc/Wildflowers700North7thAvenueTucson,AZ85705(520)622-0292TXOS/RR/HVPW/SSCompassHealthCare1230EastBroadwayStreetTucson,AZ85719(520)620-6615TXPVOS/ORIO/SSCompassHealthCare2475NorthJackRabbitStreetTucson,AZ85745(520)882-5608TXPVOSDT/ODORIORDRR/ADDDHV/SSHHCJ/FGPICopeBehavioralServices101SouthStoneStreetSuite200Tucson,AZ85701(520)884-0707TXPVOSDT/ODORIO/DDHV/SS/MDCottonwoodDeTucson4110WestSweetwaterDriveTucson,AZ85745(520)743-0411TXPVOSDT/RDRRJADDD/SSTC/FGPIDavisMonthanAirForceBaseADPB355MDOS/SGOHATucson,AZ85707-4923(520)228-4926TXPVOSDT/ODOR/DD/GH/FGHavenIncTheHaven1107EastAdelaideDriveTucson,AZ85719(520)623-4590TXOS/ORRR/SSTCLaFronteraCenterEastClinic2222NorthCraycroftRoadSuite120Tucson,AZ85712(520)296-3296x102TXPVOS/OR/ADDD/CMLaFronteraCenterHopeCenter260SouthScottStreetTucson,AZ85701(520)884-8470TXMLPVOSDT/ODOR/DDHVPW/SS/MDLaFronteraCenterSubstanceAbuseOutpatientServices502West29thStreetTucson,AZ85711(520)884-9920Hotline:(520)323-9373TXPVOS/OR

In [1026]:
rawdata = "BehavioralHealthServicesofPimaCountyInc/Wildflowers700North7thAvenueTucson,AZ85705(520)622-0292TXOS/RR/HVPW/SSCompassHealthCare1230EastBroadwayStreetTucson,AZ85719(520)620-6615TXPVOS/ORIO/SSCompassHealthCare2475NorthJackRabbitStreetTucson,AZ85745(520)882-5608TXPVOSDT/ODORIORDRR/ADDDHV/SSHHCJ/FGPICopeBehavioralServices101SouthStoneStreetSuite200Tucson,AZ85701(520)884-0707TXPVOSDT/ODORIO/DDHV/SS/MDCottonwoodDeTucson4110WestSweetwaterDriveTucson,AZ85745(520)743-0411TXPVOSDT/RDRRJADDD/SSTC/FGPIDavisMonthanAirForceBaseADPB355MDOS/SGOHATucson,AZ85707-4923(520)228-4926TXPVOSDT/ODOR/DD/GH/FGHavenIncTheHaven1107EastAdelaideDriveTucson,AZ85719(520)623-4590TXOS/ORRR/SSTCLaFronteraCenterEastClinic2222NorthCraycroftRoadSuite120Tucson,AZ85712(520)296-3296x102TXPVOS/OR/ADDD/CMLaFronteraCenterHopeCenter260SouthScottStreetTucson,AZ85701(520)884-8470TXMLPVOSDT/ODOR/DDHVPW/SS/MDLaFronteraCenterSubstanceAbuseOutpatientServices502West29thStreetTucson,AZ85711(520)884-9920Hotline:(520)323-9373TXPVOS/ORIO/DD/CMMarkYouthandFamilyCareCampusInc4653EastPimaStreetTucson,AZ85712(520)326-6182Hotlines:(520)977-7573(520)326-6182x0030TXPVOS/ORIO/AD/SSSG/FGPIPascuaYaquiChemicalDependencyProgram7490SouthCaminoDeOesteTucson,AZ85746(520)883-5000TXPVOS/ORIO/ADDD/SSPortablePrepBehavioralHealthServsAlcoholismTreatmentUnit806East46thStreetTucson,AZ85713(520)792-5706TXOS/OR/SSSaintJosephsHospitalORiellyCareCenter350NorthWilmontRoadTucson,AZ85711(520)721-6567x2528TXOS/OR10/DD/GH/MCPISalvationArmyAdultRehabilitationCenter2717South6thAvenueTucson,AZ85713(520)624-1741TXPVOS/RR/ADDD/SSTCSierraTucsonLLCLagoDelOroParkwayTucson,AZ85739(520)624-4000TXPVOSDT/IDIR/DD/PH/PITucsonAlcoholicRecoveryHomeInc1809East23rdStreetTucson,AZ85713(520)884-5180TXPVOS/RR/HHVeteransAffairsMedicalCenterSubstanceAbuseTreatmentProgram3601South6thAvenueTucson,AZ85723(520)792-1450TXPVOSDT/ODORIO/SSGH/FGPIWestcenter2105EastAllenRoadTucson,AZ85719(520)795-0952TXMLOSDT/ODIOIDIR/DD/OH/MCMDPIWHITERIVERRainbowCenterWhiteMountainApacheTribeWhiteriver,AZ85941(520)338-4858TXPVOS/ORRR/SS/FGWICKENBURGMeadowsHoldings1655NorthTegnerRoadWickenburg,AZ85390(520)684-3926TXOSDT/IDIRRDRR/DD/SSTCCMWILLIAMSTheGuidanceCenterInc301South7thStreetWilliams,AZ86046(520)635-4272TXPVOS/ORIO/ADDD/SSCMWINSLOWCommunityCounselingCentersIncOutpatientClinic211East3rdStreetWtnslow,AZ86047(520)289-4658Hotline:(520)289-4658TXPVOS/IO/DD/SSCMNavajoNationDilkonOutpatientTreatmentCenterDilkonChapterHouseWinslow,AZ86047(520)871-6235TXPVOS/IO/SSOHCMCJARIZONA12ForCodeDefinitions-SeeKEYonpagev."

In [1029]:
data = split_data(rawdata, "Tucson")
data

[['Tucson',
  ['BehavioralHealthServicesofPimaCountyInc/Wildflowers700North7thAvenueTucson,AZ85705(520)622-0292',
   'TX OS RR HV PW SS'],
  ['CompassHealthCare1230EastBroadwayStreetTucson,AZ85719(520)620-6615',
   'TX PV OS OR IO SS'],
  ['CompassHealthCare2475NorthJackRabbitStreetTucson,AZ85745(520)882-5608',
   'TX PV OS DT OD OR IO RD RR AD DD HV SS HH CJ FG PI'],
  ['CopeBehavioralServices101SouthStoneStreetSuite200Tucson,AZ85701(520)884-0707',
   'TX PV OS DT OD OR IO DD HV SS MD'],
  ['CottonwoodDeTucson4110WestSweetwaterDriveTucson,AZ85745(520)743-0411',
   'TX PV OS DT RD RR AD DD SS TC FG PI'],
  ['DavisMonthanAirForceBaseADPB355MDOS/SGOHATucson,AZ85707-4923(520)228-4926',
   'TX PV OS DT OD OR DD GH FG'],
  ['HavenIncTheHaven1107EastAdelaideDriveTucson,AZ85719(520)623-4590',
   'TX OS OR RR SS TC'],
  ['LaFronteraCenterEastClinic2222NorthCraycroftRoadSuite120Tucson,AZ85712(520)296-3296x102',
   'TX PV OS OR AD DD CM'],
  ['LaFronteraCenterHopeCenter260SouthScottStreetTucson,

In [1010]:
data = [['Montgomery',
  ['JacksonHospitalPsychiatricUnit1235ForestAvenueMontgomery,AL36106(334)293-8660',
   'TX DT ID GH MC MD FG PI'],
  ['LighthouseCounselingCenterIncIntensiveOutpatientUnit1415EastSouthBoulevardMontgomery,AL36116(334)286-5980',
   'TX PV OS OR IO DD HV PW SS MD PI'],
  ['MeadhavenAddictiveDiseaseProgram2105EastSouthBoulevardMontgomery,AL36116(334)286-3535',
   'TX PV OS DT OD IO ID IR DD GH MC FG PI'],
  ['MontgomeryMetroTreatmentCenter4303NormanBridgeRoadMontgomery,AL36105(334)288-5363',
   'TX ML PV OS DT OD OR IO SS GH SG']],
 ['Rogersville',
  ['TheFreedomHouseRoute4Rogersville,AL35652(205)247-1222',
  'TX IO RR PW SS OH HH']],
  ['Russellville',
  ['SunriseLodgeSubstanceAbuseTreatmentCenter1163WashingtonAvenueSWRussellville,AL3S653(205)332-0078',
  'TX RR SS MD PI']],
  ['Selma',
  ['CahabaCaresSubstanceAbuseServices912JeffDavisAvenueSelma,AL36701(334)874-2600Hotline:(334)875-2109',
  'TX PV OS OR IO AD HV PW SS CJ MD']],
  ['Tuskegee',
  ['CentralAlabamaVeteranHealthCareSystem2400HospitalRoadTuskegee,AL36083(334)727-0550x4184',
  'TX PV OS DT OR ID SS GH PH FG']],
  ['Muscle Shoals',
  ['ShoalsTreatmentCenter520LouiseStreetMuscleShoals,AL35661(205)383-6646',
   'TX ML PV OS DT OD OR SS PI']],
    ['Pelham',
  ['BradfordHealthServicesOakMountainShelby2280Highway35SouthPelham,AL3S124(20S)664-3480',
   'TX RR AD SS OH FG PI']],
 ['Phenix City',
  ['LifelineServicesIncRussellCountyLocation1602BroadStreetPhenixCity,AL36867(888)329-0440',
   'TX PV OS OR IO SS CJ']],
 ['Roanoke',
  ['SelfDiscoveryInc59928Highway22Roanoke,AL36274(888)283-6747',
   'TX OS DT ID IR GH MC PI']],
 ['Spanish Fort',
  ['TheShoulder4901BattleshipParkwaySpanishFort,AL36577(334)626-2199',
   'TX PV OS IO RR SS HH']],
 ['Sylacauga',
  ['CheahaRegionalMentalHealthMentalRetardationBoardlnc1721OldBirminghamHighwaySylacauga,AL351SO(256)249-2395Hotline:(256)245-2201',
   'TX PV OS DT IO RD RR AD SS MD']],
 ['Tuscaloosa',
  ['IndianRiversMentalHealthCenterAlcoholandDrugAbuseProgramS0519thAvenueTuscaloosa,AL35401(205)391-0118Hotline:(205)345-1600',
   'TX PV OS OR IO RR DD PW SS CM CH MC MD FG PI'],
  ['PhoenixHouseInc70035thAvenueTuscaloosa,AL35401(205)758-3867',
  'TX RR HH FG'],
  ['TuscaloosaTreatmentCenterS3SRiverRoadNESuiteG-3Tuscaloosa,AL35404(205)507-0800',
  'TX ML OS DT OD OR SS PI'],
  ['VeteransAffairsMedicalCenterPrimaryCareSubstanceAbuseClinic3701LoopRoadEastTuscaloosa.AL3S404(205)554-2000x2690',
  'TX PV OS DT OD OR IO SS PH FG']],
 ['Warrior',
  ['BradfordHealthServicesWarriorLodge/Jefferson1189AllbrittonRoadWarrior,AL35180(800)333-1865Hotlincs:(20S)647-1945(800)648-1945',
   'TX PV OS DT IO RD RR DD SS TC FG PI']],
 ['Wetumpka',
  ['BradfordHealthServicesatElmoreCommunityHospitalC/0ElmoreCommunityHospitalSOOHospitalDriveWetumpka,AL36092(334)567-4311',
   'TX OS DT ID IR GH']]]

In [1030]:
save_as_filename = '{0}/{0}.csv'.format(year)

with open(save_as_filename, 'a', newline='') as file:
    writer = csv.writer(file)
    city = write_data(writer, data, year, pagen)

[1998, 23, 'Behavioral Health Services of Pima County Inc/Wildflowers', '700 North 7th Avenue', 'Tucson', 'AZ', '85705', '(520)622-0292', 'TX OS RR HV PW SS']
[1998, 23, 'Compass Health Care', '1230 East Broadway Street', 'Tucson', 'AZ', '85719', '(520)620-6615', 'TX PV OS OR IO SS']
[1998, 23, 'Compass Health Care', '2475 North Jack Rabbit Street', 'Tucson', 'AZ', '85745', '(520)882-5608', 'TX PV OS DT OD OR IO RD RR AD DD HV SS HH CJ FG PI']
[1998, 23, 'Cope Behavioral Services', '101 South Stone Street Suite 200', 'Tucson', 'AZ', '85701', '(520)884-0707', 'TX PV OS DT OD OR IO DD HV SS MD']
[1998, 23, 'Cottonwood DeTucson', '4110 West Sweetwater Drive', 'Tucson', 'AZ', '85745', '(520)743-0411', 'TX PV OS DT RD RR AD DD SS TC FG PI']
[1998, 23, 'Davis Monthan Air Force BaseADPB', '355 MDOS/SGOHA', 'Tucson', 'AZ', '85707-4923', '(520)228-4926', 'TX PV OS DT OD OR DD GH FG']
[1998, 23, 'Haven Inc The Haven', '1107 East Adelaide Drive', 'Tucson', 'AZ', '85719', '(520)623-4590', 'TX OS O

- Check each term that has something blanck